In [1]:
'''
Import necessary packages
'''
import os
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
import imageio
from colormath.color_objects import sRGBColor, LabColor
from colormath.color_conversions import convert_color
from PIL import Image
import cv2
from collections import Counter

np.set_printoptions(precision=3)

%matplotlib inline

In [2]:
def convert_to_lab(arr_rgb):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_lab = []
    for row in arr_rgb:
        r, g, b = row
        list_lab.append(convert_color(sRGBColor(r, g, b, is_upscaled=True), LabColor).get_value_tuple())
    return np.array(list_lab)


def convert_to_rgb(arr_lab):
    '''
    This function will transform every rgb value in the list to lab. 
    '''
    
    list_rgb = []
    for row in arr_lab:
        l, a, b = row
        list_rgb.append(convert_color(LabColor(l, a, b), sRGBColor).get_value_tuple())
    return np.array(list_rgb)

def find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab):
    
    return arr_iscc_lab[np.argmin(cdist(arr_img_lab, arr_iscc_lab), 1)] 

# function to return key for any value 
def get_key(my_dict, val): 
    for key, value in my_dict.items(): 
         if val == value: 
                return key 
    return "key doesn't exist"
      

In [3]:
'''
Define the color card with ISCC_NBS
'''

# Read ISCC from local file
path_iscc = os.path.join('ISCC_NBS', 'ISCC_NBS_w3.csv')
df_iscc = pd.read_csv(path_iscc, names = ['color_name', 'hex'])

# Get the RGB and CIE-Lab values for each color in ISCC color card
df_iscc['RGB'] = ""
df_iscc['Lab'] = ""
for index, row in df_iscc.iterrows(): 
    df_iscc.loc[index, 'RGB'] = sRGBColor.new_from_rgb_hex(row['hex'])
    df_iscc.loc[index, 'Lab'] = convert_color(df_iscc.loc[index, 'RGB'], LabColor).get_value_tuple()

In [4]:
def img_to_vec(img_path):
    
    img = Image.open(path_img).convert('RGB')
    img = np.array(img)
    arr_img_lab = convert_to_lab(img.reshape(img.shape[0]*img.shape[1], 3))
    arr_iscc_lab = np.array(df_iscc['Lab'].tolist())
    quantized_img_lab = find_the_cloeset_iscc(arr_img_lab, arr_iscc_lab)
    quantized_img_rgb = convert_to_rgb(quantized_img_lab).reshape(img.shape[0], img.shape[1], 3)
    background_lab = np.array([95.7931405 , -0.15465805, -0.6054342])
    (unique, counts) = np.unique(quantized_img_lab, return_counts=True, axis=0)
    indices = np.argsort(counts)[::-1]
    dict_color_count = {}
    dict_color_name = {}
    sum_counts = sum(counts)


    for i in indices:
        if cdist(unique[i].reshape(1, 3), background_lab.reshape(1, 3)) < 0.001:
            continue
        index_iscc = np.where(cdist(df_iscc['Lab'].tolist(), unique[i].reshape(1, 3))<0.01)
        hex_value = df_iscc.loc[index_iscc[0][0], 'hex']
        color_name = df_iscc.loc[index_iscc[0][0], 'color_name']
        dict_color_count[hex_value] = counts[i]/sum_counts
#         dict_color_name[hex_value] = color_name
    return dict_color_count 

In [5]:
path = os.path.join('images', 'blood')
df_vec = pd.DataFrame(columns = df_iscc['hex'].values) 
for img_name in os.listdir(path):
    if img_name.startswith('.'):
        continue
        
    path_img = os.path.join(path, img_name)
    dict_color = img_to_vec(path_img)
    for color_hex in df_vec.columns:
        if color_hex not in dict_color.keys():
            dict_color[color_hex] = 0
    df_vec = df_vec.append(dict_color,ignore_index=True,sort=False)
    print(df_vec)

   #FFB5BA  #EA9399  #E4717A  #F9CCCA  #DEA5A4  #C08081  #EAD8D7  #C4AEAD  \
0      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   

   #EAE3E1  #C1B6B3  ...  #A8516E  #673147  #38152C  #AF868E  #915F6D  \
0      0.0      0.0  ...      0.0      0.0      0.0      0.0      0.0   

   #F2F3F4  #B9B8B5  #848482  #555555  #222222  
0      0.0      0.0      0.0      0.0      0.0  

[1 rows x 267 columns]
   #FFB5BA  #EA9399  #E4717A  #F9CCCA  #DEA5A4  #C08081   #EAD8D7   #C4AEAD  \
0      0.0      0.0      0.0      0.0      0.0      0.0  0.000000  0.000000   
1      0.0      0.0      0.0      0.0      0.0      0.0  0.011588  0.000047   

    #EAE3E1  #C1B6B3  ...  #A8516E  #673147  #38152C  #AF868E  #915F6D  \
0  0.000000  0.00000  ...      0.0      0.0      0.0      0.0      0.0   
1  0.004362  0.00339  ...      0.0      0.0      0.0      0.0      0.0   

   #F2F3F4  #B9B8B5  #848482  #555555   #222222  
0      0.0      0.0      0.0      0.0  0.000000  
1      0.0     

   #FFB5BA  #EA9399  #E4717A   #F9CCCA  #DEA5A4  #C08081   #EAD8D7   #C4AEAD  \
0      0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000000   
1      0.0      0.0      0.0  0.000000      0.0      0.0  0.011588  0.000047   
2      0.0      0.0      0.0  0.000000      0.0      0.0  0.000038  0.000002   
3      0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000096   
4      0.0      0.0      0.0  0.000003      0.0      0.0  0.000321  0.000309   
5      0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000023   
6      0.0      0.0      0.0  0.000007      0.0      0.0  0.000127  0.001480   
7      0.0      0.0      0.0  0.000000      0.0      0.0  0.000004  0.000000   
8      0.0      0.0      0.0  0.000000      0.0      0.0  0.000031  0.000062   

    #EAE3E1   #C1B6B3  ...  #A8516E  #673147  #38152C   #AF868E   #915F6D  \
0  0.000000  0.000000  ...      0.0      0.0      0.0  0.000000  0.000000   
1  0.004362  0.003390  ...      0.0      0.0 

    #FFB5BA  #EA9399  #E4717A   #F9CCCA  #DEA5A4  #C08081   #EAD8D7   #C4AEAD  \
0       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000000   
1       0.0      0.0      0.0  0.000000      0.0      0.0  0.011588  0.000047   
2       0.0      0.0      0.0  0.000000      0.0      0.0  0.000038  0.000002   
3       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000096   
4       0.0      0.0      0.0  0.000003      0.0      0.0  0.000321  0.000309   
5       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000023   
6       0.0      0.0      0.0  0.000007      0.0      0.0  0.000127  0.001480   
7       0.0      0.0      0.0  0.000000      0.0      0.0  0.000004  0.000000   
8       0.0      0.0      0.0  0.000000      0.0      0.0  0.000031  0.000062   
9       0.0      0.0      0.0  0.000051      0.0      0.0  0.000719  0.000193   
10      0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.001578   
11      0.0      0.0      0.

    #FFB5BA  #EA9399  #E4717A   #F9CCCA  #DEA5A4  #C08081   #EAD8D7   #C4AEAD  \
0       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000000   
1       0.0      0.0      0.0  0.000000      0.0      0.0  0.011588  0.000047   
2       0.0      0.0      0.0  0.000000      0.0      0.0  0.000038  0.000002   
3       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000096   
4       0.0      0.0      0.0  0.000003      0.0      0.0  0.000321  0.000309   
5       0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.000023   
6       0.0      0.0      0.0  0.000007      0.0      0.0  0.000127  0.001480   
7       0.0      0.0      0.0  0.000000      0.0      0.0  0.000004  0.000000   
8       0.0      0.0      0.0  0.000000      0.0      0.0  0.000031  0.000062   
9       0.0      0.0      0.0  0.000051      0.0      0.0  0.000719  0.000193   
10      0.0      0.0      0.0  0.000000      0.0      0.0  0.000000  0.001578   
11      0.0      0.0      0.

    #FFB5BA  #EA9399  #E4717A   #F9CCCA   #DEA5A4  #C08081   #EAD8D7  \
0       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
1       0.0      0.0      0.0  0.000000  0.000000      0.0  0.011588   
2       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000038   
3       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
4       0.0      0.0      0.0  0.000003  0.000000      0.0  0.000321   
5       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
6       0.0      0.0      0.0  0.000007  0.000000      0.0  0.000127   
7       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000004   
8       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000031   
9       0.0      0.0      0.0  0.000051  0.000000      0.0  0.000719   
10      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
11      0.0      0.0      0.0  0.000000  0.000000      0.0  0.065225   
12      0.0      0.0      0.0  0.000000  0.000000      0.0  0.00

    #FFB5BA  #EA9399  #E4717A   #F9CCCA   #DEA5A4  #C08081   #EAD8D7  \
0       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
1       0.0      0.0      0.0  0.000000  0.000000      0.0  0.011588   
2       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000038   
3       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
4       0.0      0.0      0.0  0.000003  0.000000      0.0  0.000321   
5       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
6       0.0      0.0      0.0  0.000007  0.000000      0.0  0.000127   
7       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000004   
8       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000031   
9       0.0      0.0      0.0  0.000051  0.000000      0.0  0.000719   
10      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
11      0.0      0.0      0.0  0.000000  0.000000      0.0  0.065225   
12      0.0      0.0      0.0  0.000000  0.000000      0.0  0.00

    #FFB5BA  #EA9399  #E4717A   #F9CCCA   #DEA5A4  #C08081   #EAD8D7  \
0       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
1       0.0      0.0      0.0  0.000000  0.000000      0.0  0.011588   
2       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000038   
3       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
4       0.0      0.0      0.0  0.000003  0.000000      0.0  0.000321   
5       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
6       0.0      0.0      0.0  0.000007  0.000000      0.0  0.000127   
7       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000004   
8       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000031   
9       0.0      0.0      0.0  0.000051  0.000000      0.0  0.000719   
10      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
11      0.0      0.0      0.0  0.000000  0.000000      0.0  0.065225   
12      0.0      0.0      0.0  0.000000  0.000000      0.0  0.00

    #FFB5BA  #EA9399  #E4717A   #F9CCCA   #DEA5A4  #C08081   #EAD8D7  \
0       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
1       0.0      0.0      0.0  0.000000  0.000000      0.0  0.011588   
2       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000038   
3       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
4       0.0      0.0      0.0  0.000003  0.000000      0.0  0.000321   
5       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
6       0.0      0.0      0.0  0.000007  0.000000      0.0  0.000127   
7       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000004   
8       0.0      0.0      0.0  0.000000  0.000000      0.0  0.000031   
9       0.0      0.0      0.0  0.000051  0.000000      0.0  0.000719   
10      0.0      0.0      0.0  0.000000  0.000000      0.0  0.000000   
11      0.0      0.0      0.0  0.000000  0.000000      0.0  0.065225   
12      0.0      0.0      0.0  0.000000  0.000000      0.0  0.00

In [6]:
df_vec.shape

(26, 267)

In [7]:
df_vec.to_csv(os.path.join(path, 'blood.csv'), index=False)